
# Por Lista Licitación

In [ ]:
import requests
import numpy as np
import pandas as pd
import re
import urllib3

urllib3.disable_warnings()

# Clave API de Mercado Público
API_KEY = "TU_API_KEY"

# Lista de códigos de Licitación
#MANUAL-> lista_lic = ["2129-20-LQ24", "3037-18-LQ24", "4237-24-LP24"]
df_lic = pd.read_excel('ListaLicitaciones.xlsx') #Subir archivo descargando lista desde Mercado Público
lista_lic = df_lic['IDLicitacion'].to_list()

# Lista para almacenar los datos
lista_datos = []

for codigo_lic in lista_lic:
    # Endpoint de la API
    url_lic = f"https://api.mercadopublico.cl/servicios/v1/publico/licitaciones.json?codigo={codigo_lic}&ticket={API_KEY}"

    # Solicitar datos de la Licitación, deshabilitando la verificación SSL
    response_lic = requests.get(url_lic, verify=False)

    if response_lic.status_code == 200:
        data_lic = response_lic.json()

        if "Listado" in data_lic and len(data_lic["Listado"]) > 0:
            lic_data = data_lic["Listado"][0]

            # Datos generales de la OC
            nombre_licitacion = lic_data.get("Nombre", "No disponible")
            comprador = lic_data.get("Comprador", {}).get("NombreOrganismo", "No disponible")
            adjudicacion = lic_data.get("Fechas", {}).get("FechaAdjudicacion", "No disponible")

            # Extraer información de los productos
            items = lic_data.get("Items", {}).get("Listado", [])

            for item in items:

                adjudicacion_data = item.get("Adjudicacion")

                #Determina si existe información de Adjudicacion o no.
                if adjudicacion_data:
                  proveedor = item.get("Adjudicacion", {}).get("NombreProveedor", "No especificado")
                  producto = item.get("NombreProducto", "No especificado")
                  descripcion = item.get("Descripcion", "No especificado")
                  cantidad = item.get("Adjudicacion", {}).get("Cantidad", 0)
                  precio = item.get("Adjudicacion", {}).get("MontoUnitario", 0)

                else:
                  proveedor = "No especificado"
                  producto = "No especificado"
                  descripcion = "No especificado"
                  cantidad = 0
                  precio = 0

                # Agregar datos a la lista
                lista_datos.append([
                    codigo_lic, nombre_licitacion, comprador, adjudicacion, proveedor,
                    producto, descripcion, cantidad, precio
                ])
        else:
            print(f"No se encontró información para la Licitación {codigo_lic}")
    else:
        print(f"Error {response_lic.status_code} en la Licitación {codigo_lic}: {response_lic.text}")

# Crear DataFrame
df = pd.DataFrame(lista_datos, columns=[
    'Codigo Licitación', 'Nombre', 'Comprador', 'Fecha Adjudicación', 'Proveedor',
    'Producto', 'Descripción', 'Cantidad', 'Precio Unitario'
])

# Mostrar las primeras filas del DataFrame
print(df.head())

# Guardar en Excel si se requiere
df.to_excel("licitaciones.xlsx", index=False)

# Por Código de Licitación

In [ ]:
import requests

# Clave API de Mercado Público
API_KEY = "TU_API_KEY"
codigo_lic = '2129-20-LQ24'
URL = f"https://api.mercadopublico.cl/servicios/v1/publico/licitaciones.json?codigo={codigo_lic}&ticket={API_KEY}"

try:
    response = requests.get(URL, timeout=30)  # 30s de espera

    if response.status_code == 200:
        data = response.json()
        print("Datos obtenidos:", data)
    else:
        print(f"Error {response.status_code}: {response.text}")

except requests.exceptions.Timeout:
    print("⌛ Error: El tiempo de espera se agotó.")
except requests.exceptions.RequestException as e:
    print(f"⚠️ Otro error: {e}")


# Por Lista de OC

In [ ]:
import requests
import numpy as np
import pandas as pd
import re

# Clave API de Mercado Público
API_KEY = "TU_API_KEY"

# Lista de códigos de órdenes de compra
lista_oc = ["979-1577-SE24", "979-1303-SE24", "979-1621-SE24", "1020-721-CM24"]  # Agrega más códigos

# Lista para almacenar los datos
lista_datos = []

# Expresiones regulares para extraer modelo y marca
modelo_pattern = r"MODELO[:\s]+([\w\s\d\-]+)"
marca_pattern = r"MARCA[:\s]+([\w\s\-]+)(?:\r|\n|$)"

for codigo_oc in lista_oc:
    # Endpoint de la API
    url_oc = f"https://api.mercadopublico.cl/servicios/v1/publico/ordenesdecompra.json?codigo={codigo_oc}&ticket={API_KEY}"

    # Solicitar datos de la OC
    response_oc = requests.get(url_oc)

    if response_oc.status_code == 200:
        data_oc = response_oc.json()

        if "Listado" in data_oc and len(data_oc["Listado"]) > 0:
            oc_data = data_oc["Listado"][0]

            # Datos generales de la OC
            codigo_licitacion = oc_data.get("CodigoLicitacion", "No disponible")
            comprador = oc_data.get("Comprador", {}).get("NombreOrganismo", "No disponible")
            proveedor = oc_data.get("Proveedor", {}).get("Nombre", "No disponible")

            # Extraer información de los productos
            items = oc_data.get("Items", {}).get("Listado", [])

            for item in items:
                producto = item.get("Producto", "No especificado")
                descripcion_comprador = item.get("EspecificacionComprador", "No especificado")
                descripcion_proveedor = item.get("EspecificacionProveedor", "No especificado")
                precio_neto = item.get("PrecioNeto", 0)

                # Extraer modelo y marca del texto ofertado por el proveedor
                modelo = re.search(modelo_pattern, descripcion_proveedor)
                marca = re.search(marca_pattern, descripcion_proveedor)

                modelo = modelo.group(1).strip() if modelo else "No especificado"
                marca = marca.group(1).strip() if marca else "No especificado"

                # Agregar datos a la lista
                lista_datos.append([
                    codigo_oc, codigo_licitacion, comprador, proveedor,
                    producto, descripcion_comprador, modelo, marca, precio_neto
                ])
        else:
            print(f"No se encontró información para la OC {codigo_oc}")
    else:
        print(f"Error {response_oc.status_code} en la OC {codigo_oc}: {response_oc.text}")

# Crear DataFrame
df = pd.DataFrame(lista_datos, columns=[
    'Codigo OC', 'Codigo Licitacion', 'Comprador', 'Proveedor',
    'Producto', 'Especificacion Comprador', 'Modelo', 'Marca', 'Precio Neto'
])

# Mostrar las primeras filas del DataFrame
print(df.head())

# Guardar en Excel si se requiere
df.to_excel("ordenes_compra.xlsx", index=False)

# Por Código OC

In [ ]:
import requests
import numpy as np
import pandas as pd
import re

# Clave API de Mercado Público
API_KEY = "TU_API_KEY"
codigo_oc = "979-1621-SE24"  # Reemplazar con código real

# Endpoint de órdenes de compra
url_oc = f"https://api.mercadopublico.cl/servicios/v1/publico/ordenesdecompra.json?codigo={codigo_oc}&ticket={API_KEY}"

# Realizar la solicitud
response_oc = requests.get(url_oc)

# Verificar respuesta
if response_oc.status_code == 200:
    data_oc = response_oc.json()
else:
    print(f"Error {response_oc.status_code}: {response_oc.text}")


codigo_oc = data_oc["Listado"][0]['Codigo']
codigo_licitacion = data_oc["Listado"][0]["CodigoLicitacion"]
comprador = data_oc["Listado"][0]['Comprador']['NombreOrganismo']
proveedor = data_oc["Listado"][0]['Proveedor']['Nombre']
features = data_oc["Listado"][0]['Items'] #Diccionario con características de la OC
q = data_oc["Listado"][0]['Items']['Cantidad'] #Cantidad de productos en OC

lista = []
for i in range(q):
  p = features['Listado'][i]['Producto'] #Nombre producto
  name = features['Listado'][i]['EspecificacionComprador'] #Lo solicitado por comprador

  texto = features['Listado'][i]['EspecificacionProveedor'] #Lo ofertado por proveeor
  # Expresiones regulares para encontrar modelo y marca
  modelo_pattern = r"MODELO:\s*([\w\s\d\-]+)"  # Busca "MODELO: ..." seguido del nombre
  marca_pattern = r"MARCA:\s*([\w\s\-]+)(?:\r|\n|$)"  # Busca "MARCA: ..." seguido del nombre
  # Extraer modelo y marca
  modelo = re.search(modelo_pattern, texto)
  marca = re.search(marca_pattern, texto)
  # Obtener los valores si existen
  modelo = modelo.group(1).strip() if modelo else "No especificado"
  marca = marca.group(1).strip() if marca else "No especificado"

  neto = features['Listado'][i]['PrecioNeto']
  lista.append([codigo_oc, codigo_licitacion, comprador, proveedor, p, name, modelo, marca, neto])

df = pd.DataFrame(lista, columns=['Codigo OC','Codigo Licitacion', 'Comprador', 'Proveedor','Producto', 'Comprador', 'Modelo', 'Marca', 'Neto'])

# Mostrar resultados
print("Codigo OC:", codigo_oc)
print("Codigo Licitacion:", codigo_licitacion)
print("Comprador:", comprador)
print("Proveedor:", proveedor)
print("Producto 1:", name)
print("Modelo 1:", modelo)
print("Marca 1:", marca)
print("Neto 1:", neto)